### Exploratory Data Analysis: FBI Crime Database
*Zach Prenovost*

First off we are going to import our method chaining function to get our reduced, usuable data set:

In [6]:
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np

def load_and_process(path_to_csv):
    
    df = (
    	pd.read_csv(path_to_csv)
    	.loc[lambda x: x['STOLEN_VALUE']>1000]
        .drop(['ORI', 'PUB_AGENCY_NAME', 'PUB_AGENCY_UNIT', 'AGENCY_TYPE_NAME', 'STATE_ABBR', 'POPULATION_GROUP_CODE', 'OFFENSE_CODE', 'VICTIM_TYPE_CODE', 'LOCATION_CODE', 'WEAPON_CODE', 'PROP_DESC_ID', 'PROP_DESC_CODE'], axis=1)
   		.sort_values("DATA_YEAR", ascending = True)
   		.rename(columns={
   			"COUNTY_NAME" : "County_Name",
   			"POPULATION_GROUP_DESC": "Population_Description", 
   			"REGION_NAME" : "Region_Name",
   			"DATA_YEAR" : "Year",
   			"STATE_NAME" : "State",
   			"DIVISION_NAME" : "Division",
   			"COUNTY_NAME" : "County",
   			"REGION_NAME" : "Region",
   			"OFFENSE_NAME" : "Offense",
   			"OFFENDER_RACE" : "Offender_Race",
   			"OFFENDER_ETHNICITY" : "Offender_Ethnicity",
   			"OFFENDER_AGE" : "Offender_Age",
   			"OFFENDER_SEX" : "Offender_Sex",
   			"VICTIM_TYPE_NAME" : "Victim_Type",
   			"LOCATION_NAME" : "Location",
   			"WEAPON_NAME" : "Weapon",
   			"PROP_DESC_NAME" : "Property_Type",
   			"STOLEN_VALUE" : "Stolen_Value",
   			"RECOVERED_VALUE" : "Recovered_Value",
   			"RECOVERED_FLAG" : "Recovered?",
   			"DATE_RECOVERED" : "Recovery_Date"
		})
    )
    
    
    return df
#run the function and save into dataframe df
df = load_and_process("../../Dataset.csv")

**So we've got our data. what now?**
lets start by looking at some of the attributes of our dataset.

In [5]:
print('shape: {}'.format(df.shape))
print('Columns: {}'.format(df.columns))

shape: (21111, 19)
Columns: Index(['Year', 'State', 'Division', 'County', 'Region',
       'Population Description', 'Offense', 'Offender Race',
       'Offender Ethnicity', 'Offender Age', 'Offender Sex', 'Victim Type',
       'Location', 'Weapon', 'Property Type', 'Stolen Value',
       'Recovered Value', 'Recovered?', 'Recovery Date'],
      dtype='object')


The `df.shape` function gives us the dimensions of our dataset. after using it we can see that there are 21,111 rows and 19 columns. 

the `df.columns` function gives us the names of all the columns included in the data set. 

Now lets look at what this dataset looks like: df.head will give us a look at the first five entries.

In [7]:
df.head()

,Year,State,Division,County,Region,Population Description,Offense,Offender Race,Offender Ethnicity,Offender Age,Offender Sex,Victim Type,Location,Weapon,Property Type,Stolen Value,Recovered Value,Recovered?,Recovery Date
45880,2013,Virginia,South Atlantic,NEWPORT NEWS CITY,South,"Cities from 100,000 thru 249,999",All Other Larceny,NaN,NaN,NaN,NaN,Individual,Parking/Drop Lot/Garage,NaN,Consumable Goods,3000.0,0.0,N,NaN
38924,2013,Tennessee,East South Central,SULLIVAN; CARTER; WASHINGTON,South,"Cities from 50,000 thru 99,999",Theft From Motor Vehicle,NaN,NaN,NaN,NaN,Business,Parking/Drop Lot/Garage,NaN,Money,2000.0,0.0,N,NaN
38736,2013,Florida,South Atlantic,MIAMI-DADE,South,"Cities from 25,000 thru 49,999",Theft From Motor Vehicle,NaN,NaN,NaN,NaN,Business,Commercial/Office Building,NaN,Radio/ TV/ VCR,6576.0,0.0,N,NaN
38214,2013,Tennessee,East South Central,SHELBY,South,"Cities from 500,000 thru 999,999",Theft From Motor Vehicle,NaN,NaN,NaN,NaN,Business,Commercial/Office Building,NaN,Radio/ TV/ VCR,1200.0,0.0,N,NaN
38213,2013,Tennessee,East South Central,SHELBY,South,"Cities from 500,000 thru 999,999",Theft From Motor Vehicle,NaN,NaN,NaN,NaN,Business,Other/Unknown,NaN,Radio/ TV/ VCR,4000.0,0.0,N,NaN


# Next we want to gain an understanding of some of the different variables in the dataset

`df.nunique` will return the count of unique values associated with each collumn, already from this we can determine a number of things about our data set

In [8]:
df.nunique(axis=0)

Year                         7
State                       40
Division                     9
County                     674
Region                       4
Population Description      17
Offense                     33
Offender Race                6
Offender Ethnicity           3
Offender Age                73
Offender Sex                 3
Victim Type                  9
Location                    44
Weapon                      12
Property Type               62
Stolen Value              2718
Recovered Value            459
Recovered?                   2
Recovery Date             1236
dtype: int64

**from this we can gather:**
* we have data entries from 7 unique years 
* 40 different states
* there were over 33 different offenses listed 
* and over 12 different options for type of weapon used

We can come back to this later when trying to answer our reasearch questions. it will be useful.

Next lets use `.describe()` to look at the count, mean, standard deviation, min, and max for the numeric variables in our data set.

In [9]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,Year,Offender Age,Stolen Value,Recovered Value
count,21111.000000,8452.000000,21111.000000,20306.000000
mean,2016.777557,32.555253,39663.855194,6778.403280
std,1.966963,12.761723,1911800.265981,37755.926278
min,2013.000000,0.000000,1001.000000,0.000000
25%,2015.000000,23.000000,2000.000000,0.000000
50%,2017.000000,31.000000,4558.000000,0.000000
75%,2019.000000,41.000000,15000.000000,1.000000
max,2019.000000,99.000000,250000000.000000,1100000.000000


Awesome! by using `df.describe` we gathered that the average value stolen was 39,663.86, the most stolen was 250,000,000 dollars. sheesh thats a lot. we also learned other things, like the average age of the offenders in our database

**now im going to look at some of the unique entries we talked about earlier, lets look at the crimes in particular!**

In [13]:
df.Offense.unique()

array(['All Other Larceny', 'Theft From Motor Vehicle', 'Robbery',
       'Theft of Motor Vehicle Parts or Accessories', 'Rape',
       'Burglary/Breaking & Entering', 'Motor Vehicle Theft',
       'Destruction/Damage/Vandalism of Property',
       'Drug/Narcotic Violations', 'Theft From Building',
       'Credit Card/Automated Teller Machine Fraud',
       'False Pretenses/Swindle/Confidence Game', 'Weapon Law Violations',
       'Kidnapping/Abduction', 'Embezzlement', 'Impersonation',
       'Simple Assault', 'Stolen Property Offenses', 'Shoplifting',
       'Murder and Nonnegligent Manslaughter', 'Intimidation',
       'Counterfeiting/Forgery', 'Wire Fraud',
       'Drug Equipment Violations', 'Extortion/Blackmail', 'Arson',
       'Identity Theft', 'Aggravated Assault',
       'Sexual Assault With An Object', 'Pocket-picking',
       'Purse-snatching', 'Pornography/Obscene Material',
       'Human Trafficking, Involuntary Servitude'], dtype=object)

we can use this function to look at all of our columns unique entries, lets look at weapons next. 

In [15]:
df.Weapon.unique()

array([nan, 'Unknown', 'None', 'Handgun', 'Handgun (Automatic)',
       'Firearm (Automatic)', 'Firearm', 'Personal Weapons', 'Rifle',
       'Knife/Cutting Instrument', 'Blunt Object', 'Other',
       'Motor Vehicle/Vessel'], dtype=object)

lets do this some more

In [21]:
df.State.unique()
print()
df."Population Description".unique()
print()

SyntaxError: invalid syntax (<ipython-input-21-bdacd8dc9dd7>, line 3)